In [23]:
import random
import imghdr
import shutil
import os
import tensorflow as tf
import pandas as pd
import numpy as np

## Data preprocessing

In [24]:
path = '/home/dell/Documentos/rn-cin-covid/'

In [25]:
def load_files(path):
    
    os.chdir(path)

    # lê arquivos .txt gerados pelo COVID-Net
    train_labels = pd.read_csv("data/train_split.txt", sep=" ",header=None)
    train_labels_gauss = pd.read_csv("data/train_labels_gauss.txt", sep=" ",header=None)
    test_labels = pd.read_csv("data/test_split.txt", sep=" ",header=None)

    # renomeia colunas
    train_labels_gauss.columns = ['id', 'image', 'label', 'database']
    train_labels.columns = ['id', 'image', 'label', 'database']
    test_labels.columns = ['id', 'image', 'label', 'database']
    
    # mantendo as colunas que importam

    train_labels_gauss = train_labels_gauss[['image','label']]
    train_labels = train_labels[['image','label']]
    test_labels = test_labels[['image','label']]
    
    return test_labels, train_labels, train_labels_gauss

def create_val(train_labels, train_labels_gauss):
    np.random.seed(97)

    val_labels = train_labels.sample(frac=0.2)
    val_labels.shape
    
    new_train_image = []
    new_train_label = []

    # separa treino de validação
    for i in range(train_labels.shape[0]):

        if train_labels['image'][i] not in val_labels['image'].tolist():
            new_train_image.append(train_labels['image'][i])
            new_train_label.append(train_labels['label'][i])


    new_train_dataset = pd.DataFrame()
    new_train_dataset['image'] = new_train_image
    new_train_dataset['label'] = new_train_label

    train_labels = new_train_dataset.copy()
    
    
    # cria equivalente com ruído
    val_labels_gauss = train_labels_gauss.loc[val_labels.index]
    
    new_train_image = []
    new_train_label = []

    # separa treino de validação com ruido
    for i in range(train_labels_gauss.shape[0]):

        if train_labels_gauss['image'][i] not in val_labels_gauss['image'].tolist():
            new_train_image.append(train_labels_gauss['image'][i])
            new_train_label.append(train_labels_gauss['label'][i])


    new_train_dataset = pd.DataFrame()
    new_train_dataset['image'] = new_train_image
    new_train_dataset['label'] = new_train_label

    train_labels_gauss = new_train_dataset.copy()

    
    return train_labels, val_labels, train_labels_gauss, val_labels_gauss

def subsampling(train_labels, val_labels, train_labels_gauss, val_labels_gauss):
    
    np.random.seed(97)

    negative_train = train_labels.loc[train_labels['label'] == 'negative'].sample(2611)
    positive_train = train_labels.loc[train_labels['label'] == 'positive']
    train_labels = pd.concat([negative_train, positive_train])

    negative_val = val_labels.loc[val_labels['label'] == 'negative'].sample(625)
    positive_val = val_labels.loc[val_labels['label'] == 'positive']
    val_labels = pd.concat([negative_val, positive_val])
    
    val_labels_gauss = val_labels_gauss.loc[val_labels.index]
    train_labels_gauss = train_labels_gauss.loc[train_labels.index]
    
    return train_labels, val_labels, train_labels_gauss, val_labels_gauss

    


In [38]:

def create_data(gauss_flag):
    
    print("Criando os dados", ' \n')
    test_labels, train_labels, train_labels_gauss = load_files(path)
    print("initial train: \n") 
    print(train_labels['label'].value_counts(), ' \n')
    print("initial train gauss: \n")
    print(train_labels_gauss['label'].value_counts(), ' \n')
    print("------------------")
    print("Validação", ' \n')
    # Validação
    train_labels, val_labels, train_labels_gauss, val_labels_gauss = create_val(train_labels, train_labels_gauss)
    print("train:\n")
    print(train_labels['label'].value_counts(), ' \n')
    print("train gauss:\n")
    print(train_labels_gauss['label'].value_counts(), ' \n')
    print("val:\n")
    print(val_labels['label'].value_counts(), ' \n')
    print("val gauss:\n")
    print(val_labels_gauss['label'].value_counts(), ' \n')
    print("------------------")
    print("Subsampling \n")
    #Subsampling
    train_labels, val_labels, train_labels_gauss, val_labels_gauss = subsampling(train_labels, val_labels, train_labels_gauss, val_labels_gauss)
    print("train:\n")
    print(train_labels['label'].value_counts(), ' \n')
    print("train gauss:\n")
    print(train_labels_gauss['label'].value_counts(), ' \n')
    print("val:\n")
    print(val_labels['label'].value_counts(), ' \n')
    print("val gauss:\n")
    print(val_labels_gauss['label'].value_counts(), ' \n')
    
    if gauss_flag == True:
        print("------------------")
        print("Concat", ' \n')
        train_labels = pd.concat([train_labels, train_labels_gauss])
        val_labels = pd.concat([val_labels,val_labels_gauss])
        
    return train_labels, val_labels, test_labels

In [39]:
train_labels, val_labels, test_labels = create_data(True)

Criando os dados  

initial train: 

negative    15068
positive     2158
Name: label, dtype: int64  

initial train gauss: 

negative    15068
positive     2158
Name: label, dtype: int64  

------------------
Validação  

train:

negative    12040
positive     1741
Name: label, dtype: int64  

train gauss:

negative    12040
positive     1741
Name: label, dtype: int64  

val:

negative    3028
positive     417
Name: label, dtype: int64  

val gauss:

negative    3028
positive     417
Name: label, dtype: int64  

------------------
Subsampling 

train:

negative    2611
positive    1741
Name: label, dtype: int64  

train gauss:

negative    2611
positive    1741
Name: label, dtype: int64  

val:

negative    625
positive    417
Name: label, dtype: int64  

val gauss:

negative    625
positive    417
Name: label, dtype: int64  

------------------
Concat  



## Movendo os arquivos para os diretórios corretos

In [25]:
os.chdir('/home/dell/Documentos/COVID-Net/data/train/') # Diretório do treino gerado pelo COVID-Net

dir_train = '/home/dell/Documentos/rn-cin-covid/data/train/' # diretório final treino
dir_test = '/home/dell/Documentos/rn-cin-covid/data/test/' # diretório final teste
dir_val = '/home/dell/Documentos/rn-cin-covid/data/val/' # diretório final validação

# removendo diretórios passados

shutil.rmtree(dir_train)
shutil.rmtree(dir_test)
shutil.rmtree(dir_val)


In [27]:
# Criando diretórios novos

for path in [dir_train, dir_test, dir_val]:
    for lb in ['positive', 'negative']:
        path_lb = os.path.join(path, lb)
        try:
            os.makedirs(path_lb)
        except OSError:
            print ("Creation of the directory %s failed" % path_lb)
        else:
            print ("Successfully created the directory %s" % path_lb)

Successfully created the directory /home/dell/Documentos/rn-cin-covid/data/train/positive
Successfully created the directory /home/dell/Documentos/rn-cin-covid/data/train/negative
Successfully created the directory /home/dell/Documentos/rn-cin-covid/data/test/positive
Successfully created the directory /home/dell/Documentos/rn-cin-covid/data/test/negative
Successfully created the directory /home/dell/Documentos/rn-cin-covid/data/val/positive
Successfully created the directory /home/dell/Documentos/rn-cin-covid/data/val/negative


In [29]:

# Copia treino e validação
for f in os.listdir():
    if str(f) in train_labels['image'].tolist():
        
        index_train = train_labels[train_labels['image']==f].index.values
        label = train_labels['label'].loc[index_train].values[0]
        shutil.copy(f, os.path.join(dir_train, label))
    
    elif str(f) in val_labels['image'].tolist():
        
        index_val = val_labels[val_labels['image']==f].index.values
        label = val_labels['label'].loc[index_val].values[0]
        shutil.copy(f, os.path.join(dir_val, label))
        

In [30]:
os.chdir('/home/dell/Documentos/COVID-Net/data/test/')


# Copia teste

for f in os.listdir():
    if str(f) in test_labels['image'].tolist():
        
        index_test = test_labels[test_labels['image']==f].index.values
        label = test_labels['label'].loc[index_test].values[0]
        shutil.copy(f, os.path.join(dir_test, label))
    else:
        print('?')
        print(f)
        


?
streptococcus-pneumoniae-pneumonia-temporal-evolution-1-day2.jpg
?
a7aef71b-0fc8-4837-be79-4ced56e03439.png
?
f31d1fb1-604b-4fec-b320-f07ce2694008.png
?
86853bbd-5af2-4551-8273-f2978dffdd76.png
?
afe1b6e2-203e-4ee5-856a-2f205d8bca2a.png
?
3db7e979-5f97-4985-bf0c-cdea7951e246.png
?
35cda03a-0898-4f8b-92d3-6f263aed23ff.png
?
fd7bad9a-1bff-49ec-9c6c-c9aae9e65726.png
?
ed0c5d1e-934a-4e31-9221-3a6a69ec9c1b.png
?
ad446933-fb8e-4739-bb40-2063e796ffd8.png
?
e2e89418-6c0e-4f50-a2d9-6cd5e58feef0.png
?
bc46651a-1314-44af-a834-1eb8a36e589e.png
?
8989e25c-a698-48fc-b428-fff56931fc8f.png
?
766b8aea-3b43-4a34-b675-09f373ca066b.png
?
e4cd65ae-65de-44fc-a6b2-ebbc46d2e8d8.png
?
eaeb935a-7294-4dd3-8bf5-73ba781d28af.png
?
91e187a5-771d-4a83-9601-e22ed326bc8e.png
?
b5234584-1487-492c-8742-444b9ca41c3d.png
?
8db71746-e837-43c2-bdf1-c44d1ab207e3.png
?
34fbff70-fa6e-4709-ab07-17f739fce394.png
?
98bfaf7a-d80b-491e-a7e0-c5c7316bebe9.png
?
d82e5841-2f43-4eab-ac17-e98a2d90c51b.png
?
6556ce72-1a60-40aa-aaf7-b50d

In [2]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
test_dir = "/home/dell/Documentos/rn-cin-covid/data/train/"

train_dataset = image_dataset_from_directory(test_dir,
                                                  shuffle=True,
                                                  labels="inferred",
                                                  label_mode="int",
                                                  batch_size=32,
                                                  image_size=(150, 150))


Found 8434 files belonging to 2 classes.


### Remove Duplicatas 

In [4]:
os.chdir('/home/dell/Documentos/rn-cin-covid/')

train_labels_gauss = pd.read_csv("data/train_labels_gauss.txt", sep=" ",header=None)
train_labels_gauss.columns = ['id', 'image', 'label', 'database']

base_dir = '/home/dell/Documentos/COVID-Net/data/'
train_dir = os.path.join(base_dir, 'train')
os.chdir(train_dir)

for image in os.listdir():
    if image in train_labels_gauss['image'].tolist():
        os.remove(image)

gauss_0bf4e2be-e95c-4e59-8cd4-db3def008067.png
igual
d5d9c073-56ac-4102-abda-ef3b1f00331b.png
gauss_99fbcbf5-17f5-43ef-8831-232c68fe72ed.png
igual
gauss_8f0cb84c-a73f-4f77-8ebb-929e961f5585.png
igual
gauss_b2521978-0df6-4fd5-b684-f03b1b96dbf7.png
igual
gauss_e0d205ec-9cff-4141-b593-eec5c0b5da21.png
igual
gauss_7bfc500f-2f54-42ab-ae52-50be710e1c08.png
igual
gauss_80ef2532-7497-4e5e-bdcc-d97da06da0b5.png
igual
b039548f-9627-4701-951b-ae89efd3789f.png
ac494c18-5513-4b25-9f80-31ce94ecf4e6.png
b1243623-70de-4b30-884c-775a3edd978e.png
33d21874-f7b1-413c-8013-0b4a98d12fb9.png
gauss_c9e3428e-c3e9-41a2-86c4-f7f489c7d40c.png
igual
0a66f58c-f98d-41d1-89af-4e0136888e13.png
gauss_7f65bfa7-4587-40e7-8acd-3c02870ab059.png
igual
669a8626-fdb4-4778-b51a-e58817d77545.png
16745_2_1.PNG
572d6aad-14be-4637-a027-4c1924c17bf8.png
gauss_DX.1.2.840.113564.1722810162.20200403113227763540.1203801020003.png
igual
b342002c-f204-47d2-b726-51e14e8dc028.png
7c0a140f-d56b-448c-8eb5-cc996fa78926.png
gauss_768e5d7f-10c8

3d8a52f8-3769-48d7-9460-2df223a06632.png
ec755d65-c3e4-427f-b7fc-7a596cf8fbb2.png
MIDRC-RICORD-1C-SITE2-000025-83262-0.png
gauss_c8697106-77a1-4a74-8843-81f8bfd4fca7.png
igual
gauss_1b6a11ed-b7e8-4c84-9ac6-8243fae589dd.png
igual
gauss_f474bb81-46a6-47d0-a554-671ea5486861.png
igual
gauss_4ccae3b7-1737-48f4-bc31-71ce2feb4b53.png
igual
ebdc4168-7cd3-477a-8a94-b768c41a9f8e.png
gauss_f2e71eff-2656-4b52-a014-eea3d0d00efc.png
igual
gauss_a940da0e-5033-4069-9318-0f675dfcfc16.png
igual
5acd71c6-2de6-453c-a801-c22aa8b4f097.png
gauss_6c5594c3-eb22-4efb-b4fa-05e3c1bb2665.png
igual
0cb6fe57-ead1-444b-8563-a51f14014e2d.png
gauss_35726244-0298-433e-9376-d1a1fe5724cf.png
igual
gauss_MIDRC-RICORD-1C-419639-001021-90730-0.png
igual
5191c1d7-b90e-46a1-b5e6-3459f03ece80.png
gauss_9bad4ae2-1652-40bf-a1bf-d6613fa70766.png
igual
8c2d4d9f-d7b4-4627-a548-e56085bbef08.png
388d9b4f-6c31-44ac-b878-8f5c3843fd1f.png
16583_1_1.jpg
f477573b-b80c-46a3-a803-b545d879475e.png
gauss_2f1f42a8-c2c1-40c1-8b62-2ffeb551e768.pn

gauss_a9883ad6-826d-4257-bd75-504e2cc7b497.png
igual
gauss_COVID(826).png
igual
gauss_b26a717e-608d-41f1-ac9a-33450bf2d7e5.png
igual
gauss_f263d63e-1715-4bd8-af5e-fa4e824fa55b.png
igual
3d340335-7343-4c65-b23e-4ff5abea13a0.png
gauss_268e70e8-ae02-4926-90fb-f458452c53d2.png
igual
98b21955-5f84-4188-8ff2-5b6bb140deb0.png
gauss_COVID(895).png
igual
gauss_1a7606e0-4d81-4bbe-ae22-93ca30540cb9.png
igual
b4ea3ed4-ba8e-4b62-8150-72e39aef9055.png
gauss_MIDRC-RICORD-1C-419639-001012-56926-1.png
igual
83bb8d1a-8486-4ee9-885d-17ddffde40fa.png
9ede7426-5aa9-4b4a-85a5-ed3ffd486640.png
gauss_7b94fc31-0735-4840-beb8-c685f22ec90b.png
igual
gauss_DX.1.2.840.113564.1722810162.202003241226081857.1203801020003.png
igual
66ab31a6-be10-4f4e-ad4d-b4aeb9371271.png
gauss_9a60bd0f-4a87-4fed-923a-7fa8ab4d9000.png
igual
23dde3cd-bdf6-4586-bf6d-8e3306c71455.png
gauss_MIDRC-RICORD-1C-419639-000906-33603-0.png
igual
9bb53dc0-28ed-47ed-bff3-9e6cc3c03d62.png
gauss_a4dbfeca-474b-4641-ab2f-7b376dea9fd8.png
igual
abe31064

23ca0450-4138-4e7a-9489-0f5b6a91031d.png
MIDRC-RICORD-1C-419639-000800-27114-0.png
gauss_916c0fd1-2ec0-42cb-a0f0-378dc882aae2.png
igual
gauss_f105d893-7962-4758-8c80-f210e2de8653.png
igual
gauss_01b9e362-4950-40f5-88fa-7557ac2a45bb.png
igual
a3914966-dba0-4178-8efa-20f3dbb5e75e.png
gauss_b501b40d-18bf-4b25-a0a4-f93634162375.png
igual
gauss_34985379-397f-4afb-9ca7-538cc629703b.png
igual
989f83d9-9e10-45e0-b197-f46ce48d7a33.png
gauss_d798aa5b-28ef-463e-81a1-c05c83dcdebb.png
igual
6d58e2bb-d792-48e1-b034-d9a304c94c99.png
gauss_8e6e3560-41b4-40ad-9cf8-0c51628bde2b.png
igual
776427e7-40a2-44df-83b0-ce646644d3aa.png
gauss_32307e74-5fa6-4aaf-acef-474a8807faec.png
igual
d4874b9e-a350-4143-9dea-8f3282605f77.png
197eb663-bd40-4c72-9fec-4160cec1f64b.png
COVID(102).png
gauss_43fe8d49-f87f-4fbe-8e6b-d715ac8905db.png
igual
ea97a1ff-8582-4f89-be4f-b8cfd2eb73b1.png
a6037427-488c-4ea1-8ea8-d81f804bd974.png
gauss_COVID(298).png
igual
6d44602a-e27d-4454-95fc-e1043d30a97d.png
gauss_81bd4aa6-fc26-42e1-a9f4

f66549b1-f756-45d1-88cd-f0083e9a25ca.png
ryct.2020003.fig2-a.png
gauss_nejmc2001573_f1b.jpeg
igual
98b52fd3-0c75-4302-b759-c731b7ca938c.png
gauss_49a26c5e-fb2f-461e-a334-e81b64fab96f.png
igual
2411eac7-f0b6-4eb8-82ec-d5f1487bdebf.png
gauss_85929859-0277-4118-8775-a4b1ca637dfe.png
igual
gauss_3c8e008b-261a-4f4e-bbb3-d3651cfd7dfa.png
igual
gauss_9d801278-8699-4eee-ac13-14de9cc773d1.png
igual
f4ad4e88-ab12-4415-b52b-f372e83b15ef.png
b6895077-3de9-4317-95a5-94950eb2ba73.png
gauss_3a9c2da0-7cf6-4d51-8012-096f6f4e461e.png
igual
9a543e59-766e-4da2-82aa-a891682b2793.png
fa892e51-e0d0-4fe8-b31e-351a8057bc84.png
gauss_52407dba-597d-4eaf-b6b8-b74ec0c1a136.png
igual
5f2b98cc-a7b5-46d5-9ea8-4fa260cb998c.png
gauss_fbf900a9-0991-45d9-8bdc-b451f2671492.png
igual
gauss_7be00aad-a51e-4e0e-bcf4-a9f620e64a4f.png
igual
gauss_34418371-837a-42d1-89fa-842508cfb742.png
igual
gauss_f5b9e64c-e373-4ccb-806c-850c5a38dad6.png
igual
3f283405-31fa-4865-acbc-0be1e297546d.png
gauss_06f4f57e-ab00-459a-aa03-2ffe1dc75bdb.

...

4fdb5a1b-7bb8-4133-aa82-2c051b94bc2e.png
gauss_MIDRC-RICORD-1C-SITE2-000252-23889-0.png
igual
gauss_f1f66038-121e-43d7-ad6f-5e17c9ffd0f2.png
igual
gauss_69145580-38bd-4aa6-8ca9-0efe12d787c6.png
igual
gauss_d21e78db-5c06-4927-b177-54c4b33144ca.png
igual
MIDRC-RICORD-1C-419639-003307-43105-0.png
gauss_e8415258-b485-47a3-a22b-758025e0793e.png
igual
f8339422-793f-40c2-8fec-34f8aa036bb0.png
5c31a6ea-f629-49e6-9115-11d1fbaf0780.png
e6817988-1f0b-4341-bc2a-99ffea80a000.png
c02e993c-7642-4d8a-aace-988c416ec09a.png
gauss_8fe3df6a-c79e-4660-b9a8-5dbe0c8922b6.png
igual
3cd75bf2-6942-4068-926d-6eadd421e303.png
gauss_86f82671-b57e-483b-a5d8-47ba057ac370.png
igual
gauss_366e8aa6-2b32-47b1-955d-1246bb000c1f.png
igual
de66cb84-3e82-4eb7-b9b8-929511f50bd6.png
MIDRC-RICORD-1C-419639-000758-13795-0.png
1738d7da-d93e-4f81-aefd-a1adb148664d.png
gauss_4ea20c39-523c-47df-a386-e32ca90ebdb0.png
igual
bc37dfed-b008-44ec-a3dd-3327b9f20be0.png
gauss_58358318-9163-40fd-a05c-e883e0e7d815.png
igual
bfea966c-f72f-42f

gauss_28775c77-80e8-4f8c-be85-7a44162b5c28.png
igual
bc2e845d-a10b-4cf8-aa3f-0ebd843b6b17.png
gauss_d9f275f9-4f05-485d-8401-810b9f07a13c.png
igual
COVID(748).png
61ac1cd2-a064-4983-8abb-9c9c06cfded8.png
gauss_e7f64666-2118-4f2f-a1e9-86d9f1eb7573.png
igual
606aba72-d0db-42d5-97e8-c40d88673dcf.png
gauss_ddb4f535-9e42-420a-9851-3fb6a06a0143.png
igual
gauss_6f316d9f-5bce-464a-997e-bb04596b9844.png
igual
f655a822-225d-4fe0-a69f-0886f91c5f2b.png
gauss_98c5dbe2-ce68-49dd-8947-3aa99e491336.png
igual
gauss_bdf9677f-6a0d-46ab-81f8-e6b64b8e839a.png
igual
gauss_d61a99d1-ed3b-49db-9e98-b57c47d289d1.png
igual
54b55fd2-35b2-47ae-80df-795c0f99f652.png
aebb193e-64ef-4368-b139-c0ee72709945.png
gauss_7c417c53-35e8-47ec-9bd1-8eb4ac0699b0.png
igual
eb81f32a-2e1b-4552-9ef6-98055b4ac687.png
gauss_9c91adc4-bb69-4e9d-a1d7-951c6fcd5f9b.png
igual
b551156f-a0e0-4895-9bd5-c4e6307f077a.png
66d6805f-88d7-4f0c-b7c5-7a702c3bd70b.png
a139ba14-0e47-46d1-aa09-b9b29b43e4c7.png
45f5be4c-3a09-42e2-a9aa-beaa6ec22039.png
4643

gauss_b76341f1-7b72-4c86-aed2-9967a0b3c729.png
igual
gauss_305506c0-6074-4612-8940-943e0e4c1bbf.png
igual
bacb8434-99c3-47e6-be98-e39fb17bf3ba.png
b9ec22f4-f273-486b-a2d4-132352dfdbc4.png
gauss_ff0090ff-4acb-4dc6-a937-86fa8f5e74da.png
igual
gauss_91dff5de-1005-4c66-8787-530f1599d8d1.png
igual
e94778df-a0c6-4d2c-93b0-7187fdeafe28.png
gauss_a1109a32-2c0d-49bc-a8fa-4efc7e46c279.png
igual
gauss_6a3a3e2d-6b76-44b0-8426-cd1fdccd2a71.png
igual
gauss_SARS-10.1148rg.242035193-g04mr34g07a-Fig7a-day5.jpeg
igual
9271f16e-e163-4ade-939b-3383b8252da3.png
gauss_f7b2bbb3-b6d2-4df7-b9d0-ba983a3c0c7c.png
igual
gauss_4ed3a113-8f50-4871-8489-1e1a17a243ab.png
igual
gauss_b0fba499-9f35-4e0d-94e5-752ed990a643.png
igual
7ad05803-8238-4bb8-bddd-365cc1381680.png
gauss_185968b1-eeb0-450a-8862-8cce9e6ed3ff.png
igual
gauss_COVID(482).png
igual
35129138-f9bb-40cb-bce0-927b0116faf7.png
gauss_2db5d817-4f4c-4773-afa0-ed3cdb3a5035.png
igual
gauss_bb51a303-4438-43ae-b843-415616c99b0d.png
igual
c2895cd9-b867-44a0-a0c4-6c

0bb1cb0e-7377-407d-845e-dce144852d32.png
gauss_b01c0d6b-11ad-463d-85bc-f4e4f1dfbc8a.png
igual
gauss_22c1b139-cb5e-41cd-83fd-7e06d6f4038e.png
igual
gauss_a5a28c3b-80c7-4741-a6d8-642329788418.png
igual
gauss_01be392f-a46d-4aef-a57e-9cd1a80dd47e.png
igual
gauss_78b8f6bc-96ba-4528-b6b7-39d1a140ce07.png
igual
gauss_COVID(493).png
igual
84da526d0453b9b9e7896631e6b366_jumbo.jpeg
4add9b3e-0e87-4875-902d-2f685166ce01.png
gauss_41d8f17e-030a-4565-9c40-2b5e9e9e993d.png
igual
cbbd4c12-de6e-4bf4-b782-0b46ca63d645.png
gauss_4dd73b83-0d4a-4755-934a-25c4fb06dc01.png
igual
gauss_54beda44-fc48-4962-9c6a-01e8001e463e.png
igual
74e774b3-ca7c-47c7-b580-393f6a432206.png
c48497e9-8a9a-4af3-92bc-cb4bbfa5f19a.png
0257438d-8365-4000-935e-3de03114734e.png
gauss_900f15a4-2f0e-45bb-bc46-29bc13723d17.png
igual
gauss_f9c26b15-b775-437a-8a40-0850794f23bb.png
igual
aefb98af-4f16-4db8-bffb-db3217f56a99.png
8a415e6e-eab0-428d-b861-8f4b3f2f657c.png
gauss_COVID(494).png
igual
686caa6d-9f3c-4a1c-87fb-84063fe336ed.png
gauss

f0ec1c0c-9c72-4edf-8c98-251a44eff025.png
9f130cae-9c08-4321-9678-349d21d14329.png
c203f4e6-0963-4543-a34c-48b9d62b3e27.png
0c5dc7d1-7022-4e46-bc02-ff9999912edd.png
be658bed-445c-4b31-a581-88ba0a008f4d.png
gauss_be7b198b-1417-4536-ac79-13dfa0529287.png
igual
aa9f772e-132a-4220-b39b-3fcd4a9eb1b6.png
gauss_1875e3ae-e7d2-41af-8305-61f7384cb19f.png
igual
99f7ad08-2189-452c-afab-6d7b1c62c049.png
96d01e60-2cfc-44f9-8b6a-f0615fa09ea5.png
gauss_ed05ddfc-2a15-4eb6-ba32-876c4a4f1f60.png
igual
4cb68d48-e377-4079-a094-2b863723204f.png
e70795e5-58b2-4b08-a9eb-6929c8905ea0.png
gauss_8e8955c9-dc5b-4109-80e8-06a714111cb0.png
igual
71ccfd03-96b3-45a5-92ec-bb4fc239a20c.png
e2a8cad0-7dd0-4d15-83ac-5414e0fe9759.png
gauss_MIDRC-RICORD-1C-419639-003235-44504-0.png
igual
b07c2ded-6e52-40de-8559-ad7ba6e246a1.png
ce8b83d0-dc04-45de-bfa1-bc67b24303a6.png
852cc5ad-522c-4a30-af8d-6dbb53fbee4d.png
ae9439c0-31a2-4b2a-946e-f9f497109c42.png
gauss_COVID(317).png
igual
gauss_f3b1b61e-693d-4fee-b67b-fa4f09554391.png
igua